# This notebook creates the dagmc.h5m for the openmc example,

It is a little verbose because I wanted to change the material tags on the h5m produced

This section makes the reactor mode, you will need jupyter-cadquery to show the model in 3d

In [ ]:
import paramak

my_reactor = paramak.SubmersionTokamak(
    inner_bore_radial_thickness=30,
    inboard_tf_leg_radial_thickness=30,
    center_column_shield_radial_thickness=30,
    divertor_radial_thickness=80,
    inner_plasma_gap_radial_thickness=50,
    plasma_radial_thickness=200,
    outer_plasma_gap_radial_thickness=50,
    firstwall_radial_thickness=3,
    blanket_rear_wall_radial_thickness=30,
    number_of_tf_coils=16,
    rotation_angle=90,
    support_radial_thickness=80,
    inboard_blanket_radial_thickness=50,
    outboard_blanket_radial_thickness=70,
    elongation=2.00,
    triangularity=0.50,
    pf_coil_radial_thicknesses=[30, 30, 30, 30],
    pf_coil_vertical_thicknesses=[30, 30, 30, 30],
    pf_coil_to_tf_coil_radial_gap=50,
    outboard_tf_coil_radial_thickness=30,
    outboard_tf_coil_poloidal_thickness=30,
    tf_coil_to_rear_blanket_radial_gap=20,
)

my_reactor.show

This section writes the default manifest file, as you can see the material tags are a bit long and hence they were shortened for the example

In [ ]:
my_reactor.export_neutronics_description(
    filename='manifest.json',
    include_sector_wedge=True
)
!cat manifest.json

simply exports all the geometry as stp files, one stp file for each component / material

In [ ]:
my_reactor.export_stp()

This is a new manifest file with simpler material tags

In [ ]:
import json

new_manifest=[
    {
        "material_tag": "tungsten",
        "stp_filename": "center_column_shield.stp",
    },
    {
        "material_tag": "steel",
        "stp_filename": "outboard_firstwall.stp",
    },
    {
        "material_tag": "flibe",
        "stp_filename": "blanket.stp",
    },
    {
        "material_tag": "tungsten",
        "stp_filename": "divertor.stp",
    },
    {
        "material_tag": "steel",
        "stp_filename": "supports.stp",
    },
    {
        "material_tag": "steel",
        "stp_filename": "outboard_rear_blanket_wall.stp",
    },
    {
        "material_tag": "copper",
        "stp_filename": "inboard_tf_coils.stp",
    },
    {
        "material_tag": "copper",
        "stp_filename": "outboard_tf_coil.stp",
    },
    {
        "material_tag": "copper",
        "stp_filename": "pf_coils.stp",
    },
    {
        "material_tag": "steel",
        "stp_filename": "pf_coil_cases.stp",
    },
    {
        "material_tag": "graveyard",
        "stp_filename": "graveyard.stp",
    },
    {
        "material_tag": "Vacuum",
        "stp_filename": "sector_wedge.stp",
        "surface_reflectivity": True,
    }
]

with open('manifest.json', 'w') as outfile:
    json.dump(new_manifest, outfile, indent=4)
!cat manifest.json

This writes out the dagmc.h5m file (assuming you have Trelis installed)

In [ ]:
paramak.neutronics_utils.trelis_command_to_create_dagmc_h5m(
        faceting_tolerance=1e-2,
        merge_tolerance=1e-4,
        material_key_name='material_tag',
        geometry_key_name='stp_filename',
        batch=True,
        manifest_filename='manifest.json',
        trelis_filename='dagmc_not_watertight.trelis',
        cubit_filename='dagmc_not_watertight.cub',
        h5m_filename='dagmc_not_watertight.h5m',
)

paramak.neutronics_utils.make_watertight(
    input_filename='dagmc_not_watertight.h5m',
    output_filename='dagmc.h5m',
)

This section might run slowly but it makes a html file that shows the model and the source particle location

In [ ]:
import openmc
source = openmc.Source()

# the distribution of radius is just a single value at the plasma major radius
radius = openmc.stats.Discrete([293.], [1])

# the distribution of source z values is just a single value
z_values = openmc.stats.Discrete([0], [1])

# the distribution of source azimuthal angles values is a uniform distribution between 0 and 2 Pi
angle = openmc.stats.Uniform(a=0., b=0.5* 3.14159265359)

# this makes the ring source using the three distributions and a radius
source.space = openmc.stats.CylindricalIndependent(r=radius, phi=angle, z=z_values, origin=(0.0, 0.0, 0.0))

# sets the direction to isotropic
source.angle = openmc.stats.Isotropic()

# sets the energy distribution to a Muir distribution neutrons
source.energy = openmc.stats.Muir(e0=14080000.0, m_rat=5.0, kt=20000.0)


neutronics_model = paramak.NeutronicsModel(
    geometry=my_reactor,
    source=source,
    materials ={
        'tungsten':'Li4SiO4',
        'steel':'Li4SiO4',
        'flibe':'Li4SiO4',
        'copper':'Li4SiO4',
    }   
)

neutronics_model.export_html(
        filename='3d_source.html',
        view_plane='XYZ',
        number_of_source_particles=1000
    )

In [ ]:
import openmc
source = openmc.Source()

# the distribution of radius is just a single value at the plasma major radius
radius = openmc.stats.Discrete([293.], [1])

# the distribution of source z values is just a single value
z_values = openmc.stats.Discrete([0], [1])

# the distribution of source azimuthal angles values is a uniform distribution between 0 and 2 Pi
angle = openmc.stats.Uniform(a=0., b=0.5* 3.14159265359)

# this makes the ring source using the three distributions and a radius
source.space = openmc.stats.CylindricalIndependent(r=radius, phi=angle, z=z_values, origin=(0.0, 0.0, 0.0))

# sets the direction to isotropic
source.angle = openmc.stats.Isotropic()

# sets the energy distribution to a Muir distribution neutrons
source.energy = openmc.stats.Muir(e0=14080000.0, m_rat=5.0, kt=20000.0)
